In [70]:
!pip install openai==0.28
!pip install dotenv
!pip install lxml_html_clean
!pip install newspaper3k
!pip install langchain_community
!pip install --upgrade langchain
import openai
import os
from langchain.prompts import ChatPromptTemplate
from newspaper import Article
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser


In [71]:
# Read the OpenAPI Key from my Google Drive
# Have your OpenAI key defined in a file such as myenv.env or your choice
# myenv.env: OPEN_API_KEY = <your open API key>
from google.colab import drive
drive.mount('/content/drive')
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('/content/drive/MyDrive/myenv.env')) # read local .env file
#openai.api_key = os.environ['OPEN_API_KEY']
#print (openai.api_key)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:

from langchain.chat_models import ChatOpenAI
llm_model = "gpt-3.5-turbo"
#llm_model = "gpt-4"
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model, openai_api_key=os.environ['OPEN_API_KEY'])


In [73]:


url = input("Enter the movie reviview article URL: ")
article = Article(url)
article.download()
article.parse()
print("\nTitle:", article.title)
print("\nText:\n", article.text)
review_text = article.text



Enter the movie reviview article URL: https://www.thehindu.com/entertainment/movies/indian-2-movie-review-kamal-haasan-shankars-underwhelming-sequel-relies-more-on-its-past-legacy-and-future-promise/article68396577.ece

Title: ‘Indian 2’ movie review: Kamal Haasan, Shankar’s underwhelming sequel relies more on its past legacy and future promise

Text:
 After making his debut with the Robin Hood-esque Gentleman and mounting a simple love story on a complex political backdrop with his sophomore film Kadhalan, director Shankar dialled it up to eleven with his third film, Indian. Apart from the trope of a former freedom fighter taking up arms against those decimating his country, the poignant backstory gave the titular hero a reason to turn vigilante, a cause worth sacrificing even his own son. This sense of emotional turmoil the character had and the empathy it rightfully extracted from us are just some of the many crucial elements that the sequel Indian 2 misses.

Senapathy (Kamal Haasan

In [74]:
review_template = """ From the text of a movie review below extract the following information \
and format the output as JSON with the below keys. Left of colons(:) below is the key and right of colons(:) is a description of that to extract: \
movie: name of the movie, \
director: if you are able to guess who the director is, \
plot: A few lines of the plot, \
cast: lead cast and crew of the movie, \
lanuage: language of the movie, \
genre: genre of the movie, \
review: summaize the review, \
reviewer: name of the reviewer, \
rating: rate the movie yourself based on the review,
good: what is positive in the movie,
bad: what is negative in the movie.
text: {text}"""


In [75]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template=' From the text of a movie review below extract the following information and format the output as JSON with the below keys. Left of colons(:) below is the key and right of colons(:) is a description of that to extract: movie: name of the movie, director: if you are able to guess who the director is, plot: A few lines of the plot, cast: lead cast and crew of the movie, lanuage: language of the movie, genre: genre of the movie, review: summaize the review, reviewer: name of the reviewer, rating: rate the movie yourself based on the review,\ngood: what is positive in the movie,\nbad: what is negative in the movie.\ntext: {text}'), additional_kwargs={})]


In [76]:
#Simple JSON Output
messages = prompt_template.format_messages(text=review_text)
chat = ChatOpenAI(temperature=0.0, model=llm_model, openai_api_key=os.environ['OPEN_API_KEY'])
response = chat(messages)
print(response.content)

{
    "movie": "Indian 2",
    "director": "Shankar",
    "plot": "Witnessing corruption and scams once again rearing their ugly heads, Senapathy returns to his homeland to finish what he started",
    "cast": ["Kamal Haasan", "Siddharth", "Rakul Preet Singh", "SJ Suryah", "Bobby Simha", "Vivek", "Priya Bhavani Shankar"],
    "language": "Tamil",
    "genre": "Action, Drama",
    "review": "Indian 2 struggles to be relevant and entertaining, lacking the grandeur and depth of its predecessor. The film falls short in terms of plot and character development.",
    "reviewer": "Unknown",
    "rating": 2,
    "good": "References to Shankar and Kamal's previous works, some moments with AR Rahman's original score",
    "bad": "Lack of depth in plot, uninteresting treatment of characters, unrealistic sequences"
}


In [77]:
#Using Structured Output Parser
review_schema = ResponseSchema(name="review",
                             description="summarize the review")
reviewer_schema = ResponseSchema(name="reviewer",
                             description="name of the reviewer")
movie_schema = ResponseSchema(name="movie",
                             description="name of the movie")
director_schema = ResponseSchema(name="director",
                             description="if you are able to guess who the director is")
plot_schema = ResponseSchema(name="plot",
                             description="A few lines of the plot")
cast_schema = ResponseSchema(name="cast",
                             description="lead cast and crew of the movie")
language_schema = ResponseSchema(name="language",
                             description="language of the movie")
genre_schema = ResponseSchema(name="genre",
                             description="genre of the movie")
rating_schema = ResponseSchema(name="rating",
                             description="rate the movie yourself based on the review")
good_schema = ResponseSchema(name="good",
                             description="what is positive in the movie")
bad_schema = ResponseSchema(name="bad",
                             description="what is negative in the movie")
response_schemas = [review_schema,
                    reviewer_schema,
                    movie_schema,
                    director_schema,
                    plot_schema,
                    cast_schema,
                    language_schema,
                    genre_schema,
                    rating_schema,
                    good_schema,
                    bad_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"review": string  // summarize the review
	"reviewer": string  // name of the reviewer
	"movie": string  // name of the movie
	"director": string  // if you are able to guess who the director is
	"plot": string  // A few lines of the plot
	"cast": string  // lead cast and crew of the movie
	"language": string  // language of the movie
	"genre": string  // genre of the movie
	"rating": string  // rate the movie yourself based on the review
	"good": string  // what is positive in the movie
	"bad": string  // what is negative in the movie
}
```


In [78]:
review_template = """ From the text of a movie review below output the following information in JSON \
text: {text}
{format_instructions}"""

prompt_template = ChatPromptTemplate.from_template(review_template)
messages = prompt_template.format_messages(text=review_text, format_instructions=format_instructions)
print(messages[0].content)
response = chat(messages)
print(response.content)

#Convert the response into a python dictionary

output_dict = output_parser.parse(response.content)
output_dict
type(output_dict)

print(f"Movie rating is: {output_dict.get('rating')}")


 From the text of a movie review below output the following information in JSON text: After making his debut with the Robin Hood-esque Gentleman and mounting a simple love story on a complex political backdrop with his sophomore film Kadhalan, director Shankar dialled it up to eleven with his third film, Indian. Apart from the trope of a former freedom fighter taking up arms against those decimating his country, the poignant backstory gave the titular hero a reason to turn vigilante, a cause worth sacrificing even his own son. This sense of emotional turmoil the character had and the empathy it rightfully extracted from us are just some of the many crucial elements that the sequel Indian 2 misses.

Senapathy (Kamal Haasan), who reaches Hong Kong at the end of Indian, is now in Taiwan, the land where his idol Subhas Chandra Bose died. While those who waited for Bose’s return never got to see that, a bunch of YouTubers, disillusioned by corruption, trend #ComeBackIndian on social media c